# 📘 1강: CNN 기초 실습 — CIFAR-10 분류

**목표**
- 이미지 데이터 구조를 이해하고, CIFAR-10 데이터셋으로 **CNN을 처음부터 학습**합니다.
- 전처리 → 모델구축 → 학습 → 평가 → **오분류 분석** → **성능개선(증강/드롭아웃)** 순으로 실습합니다.

**소요시간 가이드 (30분)**
- 도입/데이터 탐색(10분), 모델 구축/학습(10분), 평가/개선(10분)


## 0. 환경 설정 및 라이브러리 임포트

In [ ]:
## ✨ 환경 체크 및 라이브러리 임포트
import os, sys, time
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import confusion_matrix, classification_report

print(f"TensorFlow: {tf.__version__}")
print("GPU Available:", tf.config.list_physical_devices('GPU'))


## 1. 데이터셋 로드 및 탐색 (CIFAR-10)

In [ ]:
## 📦 CIFAR-10 데이터셋 로드
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
## 🔍 샘플 이미지 그리드 확인 (5x5)
plt.figure(figsize=(8,8))
idx = np.random.choice(len(x_train), 25, replace=False)
for i, id_ in enumerate(idx, 1):
    plt.subplot(5,5,i)
    plt.imshow(x_train[id_])
    plt.title(class_names[y_train[id_]])
    plt.axis('off')
plt.tight_layout(); plt.show()

In [ ]:
## 📊 클래스 분포 확인 (train)
counts = np.bincount(y_train, minlength=10)
plt.figure(figsize=(8,3))
plt.bar(range(10), counts)
plt.xticks(range(10), class_names, rotation=45)
plt.title('Class Distribution (Train)')
plt.tight_layout(); plt.show()

In [ ]:
## 📈 픽셀 분포 확인 (정규화 전)
plt.figure(figsize=(6,3))
plt.hist(x_train.ravel(), bins=50)
plt.title('Raw Pixel Value Distribution (0~255)')
plt.tight_layout(); plt.show()

## 2. 데이터 전처리 (정규화, 검증셋 분리)

In [ ]:
## ✂️ Train/Validation 분리 (8:2), 0~1 정규화
from sklearn.model_selection import train_test_split

x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32')  / 255.0

x_tr, x_va, y_tr, y_va = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)
x_tr.shape, x_va.shape, y_tr.shape, y_va.shape

## 3. (Baseline) Dense-Only 모델로 비교하기

In [ ]:
## 📐 Dense 네트워크 (Conv 없이 비교용)
mlp = models.Sequential([
    layers.Flatten(input_shape=(32,32,3)),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])
mlp.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
hist_mlp = mlp.fit(x_tr, y_tr, validation_data=(x_va, y_va), epochs=5, batch_size=128, verbose=1)

## 4. CNN 모델 구성 (V1: 기본 구조)

In [ ]:
## 🧱 간단한 CNN 모델
def build_cnn_v1() -> tf.keras.Model:
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.Flatten(),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

cnn_v1 = build_cnn_v1()
cnn_v1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## 🛑 과적합 방지 콜백
ckpt_path = 'cnn_v1_best.keras'
callbacks=[
    EarlyStopping(patience=3, restore_best_weights=True, monitor='val_accuracy'),
    ModelCheckpoint(ckpt_path, monitor='val_accuracy', save_best_only=True)
]
hist_v1 = cnn_v1.fit(x_tr, y_tr, validation_data=(x_va, y_va), epochs=15, batch_size=128, callbacks=callbacks, verbose=1)

In [ ]:
## 📉 학습 곡선 시각화 함수
def plot_history(history, title='Training History'):
    h = history.history
    plt.figure(figsize=(6,4))
    plt.plot(h['accuracy'], label='train_acc')
    plt.plot(h['val_accuracy'], label='val_acc')
    plt.title(title)
    plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.tight_layout(); plt.show()
    
    plt.figure(figsize=(6,4))
    plt.plot(h['loss'], label='train_loss')
    plt.plot(h['val_loss'], label='val_loss')
    plt.title(title)
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.tight_layout(); plt.show()

plot_history(hist_mlp, 'MLP History')
plot_history(hist_v1, 'CNN V1 History')

## 5. 테스트 평가 및 혼동행렬

In [ ]:
## 🧪 테스트 평가
test_loss, test_acc = cnn_v1.evaluate(x_test, y_test, verbose=0)
print(f"[CNN V1] Test Acc: {test_acc:.4f}  |  Test Loss: {test_loss:.4f}")

## 🔢 예측 및 혼동행렬
y_pred = np.argmax(cnn_v1.predict(x_test, verbose=0), axis=1)
cm = confusion_matrix(y_test, y_pred, labels=range(10))

plt.figure(figsize=(6,5))
plt.imshow(cm, interpolation='nearest')
plt.title('Confusion Matrix')
plt.colorbar(); plt.xticks(range(10), class_names, rotation=45); plt.yticks(range(10), class_names)
plt.tight_layout(); plt.xlabel('Predicted'); plt.ylabel('True'); plt.show()

print('\n[Classification Report]\n')
print(classification_report(y_test, y_pred, target_names=class_names))

## 6. 오분류 사례 분석 (에러 분석)

In [ ]:
mis_idx = np.where(y_pred != y_test)[0]
print(f"오분류 개수: {len(mis_idx)}")
plt.figure(figsize=(8,8))
for i, id_ in enumerate(mis_idx[:16], 1):
    plt.subplot(4,4,i)
    plt.imshow(x_test[id_])
    plt.title(f"T:{class_names[y_test[id_]]}\nP:{class_names[y_pred[id_]]}")
    plt.axis('off')
plt.tight_layout(); plt.show()

## 7. 성능 개선 — 데이터 증강 & 드롭아웃 (V2)

In [ ]:
## 🔁 데이터 증강 파이프라인
data_aug = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

## 🧱 CNN V2: 증강 + 드롭아웃 확장
def build_cnn_v2() -> tf.keras.Model:
    inputs = layers.Input(shape=(32,32,3))
    x = data_aug(inputs)
    x = layers.Conv2D(32, (3,3), activation='relu')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(64, (3,3), activation='relu')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(128, (3,3), activation='relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

cnn_v2 = build_cnn_v2()
cnn_v2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
hist_v2 = cnn_v2.fit(x_tr, y_tr, validation_data=(x_va, y_va), epochs=15, batch_size=128, verbose=1)

## ✅ 테스트 성능 비교
acc_v1 = cnn_v1.evaluate(x_test, y_test, verbose=0)[1]
acc_v2 = cnn_v2.evaluate(x_test, y_test, verbose=0)[1]
print(f"V1 Test Acc: {acc_v1:.4f} | V2 Test Acc: {acc_v2:.4f}")

plot_history(hist_v2, 'CNN V2 History')

## 8. 모델 저장 및 로딩

In [ ]:
cnn_v2.save('cnn_v2_final.keras')
print('모델 저장 완료: cnn_v2_final.keras')

loaded = tf.keras.models.load_model('cnn_v2_final.keras')
print('로딩 테스트:', loaded.evaluate(x_test, y_test, verbose=0))

### ✅ 요약
- CNN은 이미지의 지역적 패턴을 자동 학습하여 분류 성능을 높입니다.
- 데이터 증강과 드롭아웃을 통해 **과적합을 완화**하고 일반화 성능을 높일 수 있습니다.
- 다음 강의에서는 **사전학습(Transfer Learning)** 으로 더 빠르고 높은 성능을 달성해 봅니다.
